### Домашнее задание

Нужно реализовать rest api на базе flask (пример https://github.com/fimochka-sudo/GB_docker_flask_example)

По шагам:
0. выбрать себе датасет (который интересен или нравится больше всего), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
1. установить удобную для себя среду разработки (pycharm прекрасен - https://www.jetbrains.com/pycharm/)
2. для вашего проекта вам понадобится requirements.txt с пакетами. Можно за основу взять такой файл из проекта выше. Для его установки прям в pycharm можно открыть терминал и сделать pip install -r requirements.txt (находясь в корне проекта конечно же при этом)
3. завести себе аккаунт на github (если его еще нет). У самого github есть такой "hello world" по работе с ним - https://guides.github.com/activities/hello-world/
4. итоговый проект должен содержать: 1) каталог app/models/ (здесь модель-пайплайн предобученная либо код обучения модели-пайплайна) 2) файл app/run_server.py (здесь основной код flask-приложения) 3) requirements.txt (список пакетов, которые у вас используются в проекте - в корне проекта) 4) README.md (здесь какое-то описание, что вы делаете, что за данные, как запускать и т.д) 5) Dockerfile 6) docker-entrypoint.sh
5. (<b>Опционально</b>): front-end сервис какой-то, который умеет принимать от пользователя введеные данные и ходить в ваш api. На самом деле полезно больше вам, т.к если ваш проект будет далее развиваться (новые модели, интересные подходы), то это хороший пунктик к резюме и в принципе - строчка в портфолио)


In [1]:
!pip install dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import dill
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
#normalizing data
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score
#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

In [3]:
! pip install kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d kukuroo3/body-signal-of-smoking

  0% 0.00/3.40M [00:00<?, ?B/s]
100% 3.40M/3.40M [00:00<00:00, 44.7MB/s]


In [4]:
! unzip body-signal-of-smoking.zip

Archive:  body-signal-of-smoking.zip
  inflating: competition_format/x_test.csv  
  inflating: competition_format/x_train.csv  
  inflating: competition_format/y_test.csv  
  inflating: competition_format/y_train.csv  
  inflating: smoking.csv             


#Будем определять являяется ли пациент курильщиком (доля вероятности)

#Список признаков

* ID : index
* gender
* age : 5-years gap
* height(cm)
* weight(kg)
* waist(cm) : Waist circumference length
* eyesight(left)
* eyesight(right)
* hearing(left)
* hearing(right)
* systolic : Blood pressure
* relaxation : Blood pressure
* fasting blood sugar
* Cholesterol : total
* triglyceride
* HDL : cholesterol type
* LDL : cholesterol type
* hemoglobin
* Urine protein
* serum creatinine
* AST : glutamic oxaloacetic transaminase type
* ALT : glutamic oxaloacetic transaminase type
* Gtp : γ-GTP
* oral : Oral Examination status
* dental caries
* tartar : tartar status
* smoking

#Для последующей работы отберём:

Категориальные признаки:
* gender
* tartar : tartar status

Вещественные признаки:
* age : 5-years gap
* height(cm)
* triglyceride
* hemoglobin
* Gtp : γ-GTP
* dental caries

In [5]:
df = pd.read_csv('smoking.csv')
df.drop(columns='ID', inplace=True)
df

,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,oral,dental caries,tartar,smoking
0,F,40,155,60,81.3,1.2,1.0,1.0,1.0,114.0,...,12.9,1.0,0.7,18.0,19.0,27.0,Y,0,Y,0
1,F,40,160,60,81.0,0.8,0.6,1.0,1.0,119.0,...,12.7,1.0,0.6,22.0,19.0,18.0,Y,0,Y,0
2,M,55,170,60,80.0,0.8,0.8,1.0,1.0,138.0,...,15.8,1.0,1.0,21.0,16.0,22.0,Y,0,N,1
3,M,40,165,70,88.0,1.5,1.5,1.0,1.0,100.0,...,14.7,1.0,1.0,19.0,26.0,18.0,Y,0,Y,0
4,F,40,155,60,86.0,1.0,1.0,1.0,1.0,120.0,...,12.5,1.0,0.6,16.0,14.0,22.0,Y,0,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55687,F,40,170,65,75.0,0.9,0.9,1.0,1.0,110.0,...,12.3,1.0,0.6,14.0,7.0,10.0,Y,1,Y,0
55688,F,45,160,50,70.0,1.2,1.2,1.0,1.0,101.0,...,14.0,1.0,0.9,20.0,12.0,14.0,Y,0,Y,0
55689,F,55,160,50,68.5,1.0,1.2,1.0,1.0,117.0,...,12.4,1.0,0.5,17.0,11.0,12.0,Y,0,N,0
55690,M,60,165,60,78.0,0.8,1.0,1.0,1.0,133.0,...,14.4,1.0,0.7,20.0,19.0,18.0,Y,0,N,0


In [6]:
#отберём данные для последующей проверки нашей модели через Flask
user_data = df.drop(columns='smoking')[:10]

In [62]:
#выберем наиболее важные признаки
categorical_columns = ['gender', 'tartar']
continuous_columns = ['age', 'height(cm)', 'triglyceride', 'hemoglobin', 'Gtp', 'dental caries']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='smoking')[10:], df['smoking'][10:], test_size=0.3)

X_train = X_train[categorical_columns + continuous_columns]
X_test = X_test[categorical_columns + continuous_columns]
user_data = user_data[categorical_columns + continuous_columns]

#save test
X_test.to_csv("/content/drive/MyDrive/X_test.csv", index=None)
y_test.to_csv("/content/drive/MyDrive/y_test.csv", index=None)
#save train
X_train.to_csv("/content/drive/MyDrive/X_train.csv", index=None)
y_train.to_csv("/content/drive/MyDrive/y_train.csv", index=None)
#save user_data
user_data.to_csv('/content/drive/MyDrive/user_data.csv', index=None)

In [9]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]

class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

class Outliers(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X.loc[X[self.key] > np.quantile(X[self.key], q=0.995), self.key] = X[self.key].mode()
        X.loc[X[self.key] < np.quantile(X[self.key], q=0.005), self.key] = X[self.key].mode()
        return X


In [10]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', NumberSelector(key=cat_col)),
                ('onehot', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))

for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

feats = FeatureUnion(final_transformers)

In [11]:
feature_processing = Pipeline([('feats', feats)])

pipeline = Pipeline([
    ('feats',feats),
    ('classifier', xgb.XGBClassifier(n_estimators=650, max_depth=10, learning_rate=0.02, min_split_loss=1)),
])

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('feats',
                 FeatureUnion(transformer_list=[('gender',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='gender')),
                                                                 ('onehot',
                                                                  OHEEncoder(key='gender'))])),
                                                ('tartar',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tartar')),
                                                                 ('onehot',
                                                                  OHEEncoder(key='tartar'))])),
                                                ('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('scaler',
                                                                  Standard...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=0.02,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=10, max_leaves=None,
                               min_child_weight=None, min_split_loss=1,
                               missing=nan, monotone_constraints=None,
                               n_estimators=650, n_jobs=None,
                               num_parallel_tree=None, predictor=None, ...))])

In [13]:
#наши прогнозы для тестовой выборки
preds = pipeline.predict_proba(X_test)[:, 1]
preds[:10]

array([0.6723116 , 0.7098215 , 0.7568867 , 0.04039266, 0.7575137 ,
       0.11146685, 0.02178748, 0.02794814, 0.0218776 , 0.8298757 ],
      dtype=float32)

Метрики качества модели

In [14]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

precision, recall, thresholds = precision_recall_curve(y_test, preds)
fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix],
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.371682, F-Score=0.722, Precision=0.610, Recall=0.883


In [15]:
#сохранение модели
with open("/content/drive/MyDrive/XGBClass_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

Передача запросов

In [20]:
import requests
import urllib.request
import json

In [72]:
#Формирование запроса
def send_json(x):

  body = {"gender": [i for i in x['gender']],
          "tartar": [i for i in x['tartar']],
          "age": [i for i in x['age']],
          "height(cm)": [i for i in x['height(cm)']],
          "triglyceride": [i for i in x['triglyceride']],
          "hemoglobin": [i for i in x['hemoglobin']],
          "Gtp": [i for i in x['Gtp']],
          "dental caries": [i for i in x['dental caries']]}

  myurl = 'http://ebca-34-125-0-115.ngrok-free.app' + '/predict'
  headers = {'content-type': 'application/json; charset=utf-8'}

  response = requests.post(myurl, json=body, headers=headers)
  response.raise_for_status()  # raises exception when not a 2xx response
  if response.status_code != 204:
    return response.json()['predictions']


In [91]:
response =  send_json(user_data)
for i, j in enumerate(response):
  print(f'Вероятность того, что пациент {i} является курильщиком составляет {round(float(j), 2)*100} %')

Вероятность того, что пациент 0 является курильщиком составляет 12.0 %
Вероятность того, что пациент 1 является курильщиком составляет 4.0 %
Вероятность того, что пациент 2 является курильщиком составляет 44.0 %
Вероятность того, что пациент 3 является курильщиком составляет 57.99999999999999 %
Вероятность того, что пациент 4 является курильщиком составляет 2.0 %
Вероятность того, что пациент 5 является курильщиком составляет 44.0 %
Вероятность того, что пациент 6 является курильщиком составляет 73.0 %
Вероятность того, что пациент 7 является курильщиком составляет 75.0 %
Вероятность того, что пациент 8 является курильщиком составляет 2.0 %
Вероятность того, что пациент 9 является курильщиком составляет 63.0 %
